## Lab 11


In [60]:
import string 
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report

In [61]:
df = pd.read_csv('DataSets\DataSet13_1.csv')
df.head()

,id,text,author
0,0,-Бабушка!- вскричала малютка.- Возьми меня с с...,Dostoevsky
1,1,"Знал ли Скрудж об этом? Разумеется, знал. Да и...",Dostoevsky
2,2,"-С праздником, дядя, с радостью! Дай вам Бог в...",Dostoevsky
3,3,Мы высказали только главную передовую мысль на...,Dostoevsky
4,4,"I. Отдел литературный. Повести, романы, расска...",Dostoevsky


In [62]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(['id', 'author'], axis=1), df.author,test_size=0.2, random_state= 42)

In [63]:
vectorizer = CountVectorizer()
vectorizer.fit(pd.concat([x_test.text,x_train.text], ignore_index= True))
X_test = vectorizer.transform(x_test.text)
X_train = vectorizer.transform(x_train.text)
Y_train = y_train.values
Y_test = y_test.values

## Log reg

In [64]:
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
pred = log_reg.predict(X_test)
print(f'Acc score: {accuracy_score(Y_test,pred)}')
print('Report:\n',classification_report(Y_test, pred))

Acc score: 0.792507204610951
Report:
               precision    recall  f1-score   support

      Akunin       0.76      0.94      0.84       106
    Bulychev       0.88      0.77      0.82        56
      Chehov       0.71      0.69      0.70        29
  Dostoevsky       0.60      0.21      0.32        14
       Gogol       0.53      0.41      0.46        22
        King       0.86      0.85      0.86        60
   Pratchett       0.83      0.88      0.85        49
      Remark       1.00      0.55      0.71        11

    accuracy                           0.79       347
   macro avg       0.77      0.66      0.69       347
weighted avg       0.79      0.79      0.78       347



c:\Users\UAZS8\Desktop\PYTHON\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Rand forest

In [65]:
rnd_forest = RandomForestClassifier()##n_estimators= 100, criterion= 'log_loss', min_samples_split= 1000)
rnd_forest.fit(X_train, Y_train)
pred = rnd_forest.predict(X_test)
print(f'Acc score: {accuracy_score(Y_test,pred)}')
print('Report:\n',classification_report(Y_test, pred))

Acc score: 0.6080691642651297
Report:
               precision    recall  f1-score   support

      Akunin       0.52      1.00      0.68       106
    Bulychev       0.82      0.66      0.73        56
      Chehov       0.60      0.21      0.31        29
  Dostoevsky       0.33      0.07      0.12        14
       Gogol       0.30      0.14      0.19        22
        King       0.78      0.63      0.70        60
   Pratchett       0.76      0.39      0.51        49
      Remark       1.00      0.09      0.17        11

    accuracy                           0.61       347
   macro avg       0.64      0.40      0.43       347
weighted avg       0.65      0.61      0.57       347



## Обработка

In [77]:
from nltk.stem import WordNetLemmatizer

In [78]:
lemm = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemm.lemmatize(word) for word in text.split()])

In [86]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
STOP_WORDS = set(stopwords.words('russian'))
def remove_stopwords(text):
    return ' '.join([word for word in str(text).split() if word not in STOP_WORDS])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\UAZS8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\UAZS8\AppData\Roaming\nltk_data...


In [87]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punct(text):
    return text.translate(str.maketrans('','', PUNCT_TO_REMOVE))

In [88]:
train_preprocessed = {'text':[], 'author': []}
train_preprocessed = pd.DataFrame(train_preprocessed)

for index, row in x_train.iterrows():
    new_row = pd.DataFrame({'text': lemmatize_words(remove_stopwords(remove_punct(row['text']))), 'author': y_train[index]}, index = [index])
    train_preprocessed = pd.concat([train_preprocessed, new_row]). reset_index(drop = True)
    
test_preprocessed = {'text':[], 'author': []}
test_preprocessed = pd.DataFrame(test_preprocessed)

for index, row in x_test.iterrows():
    new_row = pd.DataFrame({'text': lemmatize_words(remove_stopwords(remove_punct(row['text']))), 'author': y_test[index]}, index = [index])
    test_preprocessed = pd.concat([test_preprocessed, new_row]).reset_index(drop = True)

In [89]:
x_train = train_preprocessed.drop(['author'], axis = 1)
y_train = train_preprocessed.author
x_test = test_preprocessed.drop(['author'], axis = 1)
y_test = test_preprocessed.author

In [90]:
vectorizer = CountVectorizer()
vectorizer.fit(pd.concat([x_test.text,x_train.text], ignore_index= True))
X_test = vectorizer.transform(x_test.text)
X_train = vectorizer.transform(x_train.text)
Y_train = y_train.values
Y_test = y_test.values

In [91]:
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
pred = log_reg.predict(X_test)
print(f'Acc score: {accuracy_score(Y_test,pred)}')
print('Report:\n',classification_report(Y_test, pred))

Acc score: 0.7204610951008645
Report:
               precision    recall  f1-score   support

      Akunin       0.79      0.87      0.83       106
    Bulychev       0.85      0.84      0.85        56
      Chehov       0.83      0.52      0.64        29
  Dostoevsky       0.00      0.00      0.00        14
       Gogol       0.50      0.41      0.45        22
        King       0.90      0.62      0.73        60
   Pratchett       0.48      0.88      0.62        49
      Remark       0.88      0.64      0.74        11

    accuracy                           0.72       347
   macro avg       0.65      0.60      0.61       347
weighted avg       0.73      0.72      0.71       347



c:\Users\UAZS8\Desktop\PYTHON\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\UAZS8\Desktop\PYTHON\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UAZS8\Desktop\PYTHON\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score